In [2]:
import ollama
import os
import json
import re
import random
from src.rationalization import rationalize  # Ensure this module is accessible

# Define the path to the incorrect_pairs.json file
NUMBER_OF_PAIRS_TO_PROCESS = 4
INCORRECT_PAIRS_FILE = 'data/incorrect_pairs.json'
OLLOMA_MODEL_NAME = "llama-reason-05:latest"

# Initialize lists to store categorized pairs
correct_pairs = []
unanswered_pairs = []
unmatched_pairs = []

def load_incorrect_pairs(file_path):
    """
    Loads incorrect pairs from a JSON file.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        list: List of incorrect pairs.
    """
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return []

    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            data = json.load(file)
            if isinstance(data, list):
                return data
            else:
                print(f"Unexpected data format in {file_path}. Expected a list.")
                return []
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {file_path}: {e}")
            return []

def save_results(data, file_path):
    """
    Saves categorized pairs to a specified JSON file.

    Args:
        data (list): List of categorized pairs.
        file_path (str): Path to the output JSON file.
    """
    if not data:
        print(f"No data to save for {file_path}.")
        return

    # Check if the file exists; if not, create it with an empty list
    if not os.path.exists(file_path):
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump([], file, ensure_ascii=False, indent=4)

    # Load existing data
    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            existing_data = json.load(file)
            if not isinstance(existing_data, list):
                raise ValueError("Data is not a list")
        except (json.JSONDecodeError, ValueError) as e:
            print(f"Error reading {file_path}: {e}. Please check the file format.")
            return

    # Append new data
    existing_data.extend(data)

    # Save back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

    print(f"Saved {len(data)} pair(s) to {file_path}.")

def extract_decision(response_content):
    """
    Extracts 'correct' or 'incorrect' from the model's response.

    Args:
        response_content (str): The raw response from the model.

    Returns:
        str: 'correct', 'incorrect', or '' if not found.
    """
    # Define the regex pattern to match 'correct' or 'incorrect' as whole words
    pattern = r'\b(correct|incorrect)\b'
    match = re.search(pattern, response_content.lower())
    if match:
        return match.group(1)
    return ''

def extract_final_answer(rationale):
    """
    Extracts the final answer from the rationale.

    Args:
        rationale (str): The generated rationale.

    Returns:
        str: The extracted answer or an empty string if not found.
    """
    match = re.search(r'Answer:\s*(.*)', rationale, re.IGNORECASE)
    if match:
        return match.group(1).strip().lower()
    return ''

def compare_answers(extracted, expected, margin=1.0):
    """
    Compares the extracted answer with the expected answer numerically.
    Allows for minor discrepancies due to approximations.

    Args:
        extracted (str): The extracted answer from the rationale.
        expected (str): The expected correct answer.
        margin (float): Allowed margin of error.

    Returns:
        bool: True if answers match within the margin, False otherwise.
    """
    try:
        # Extract numerical values
        extracted_num = float(re.findall(r'\d+\.?\d*', extracted)[0])
        expected_num = float(re.findall(r'\d+\.?\d*', expected)[0])
        # Allow a small margin for approximation
        return abs(extracted_num - expected_num) <= margin
    except (IndexError, ValueError):
        return False

def process_incorrect_pairs(num_pairs, model_name="llama-reason-05:latest"):
    """
    Processes a specified number of incorrect pairs with combined evaluation.

    Args:
        num_pairs (int): Number of pairs to process.
        model_name (str): Name of the model to use with Ollama.
    """
    # Load incorrect pairs from the JSON file
    incorrect_pairs = load_incorrect_pairs(INCORRECT_PAIRS_FILE)

    if not incorrect_pairs:
        print("No incorrect pairs to process.")
        return

    # Determine the actual number of pairs to process
    num_to_process = min(num_pairs, len(incorrect_pairs))
    print(f"Processing {num_to_process} out of {len(incorrect_pairs)} incorrect pair(s).")

    # Select num_to_process random pairs
    pairs_to_process = random.sample(incorrect_pairs, num_to_process)

    for idx, pair in enumerate(pairs_to_process, 1):
        question = pair.get('question', '').strip()
        correct_answer = pair.get('correct_answer', '').strip()

        if not question or not correct_answer:
            print(f"Skipping pair {idx} due to missing question or correct_answer.")
            continue

        print(f"\nProcessing Pair {idx}:")
        print(f"Question: {question}")
        print(f"Correct Answer: {correct_answer}")

        attempts = 0
        max_attempts = 1  # Total attempts: initial + one retry
        pair_correct = False

        while attempts < max_attempts and not pair_correct:
            attempts += 1
            print(f"Attempt {attempts} for Pair {idx}.")

            # Generate the rationale with the correct answer as a hint
            generated_rationale = rationalize(question, correct_answer)

            # Extract the final answer from the rationale
            extracted_answer = extract_final_answer(generated_rationale)

            # If programmatic evaluation fails, use model-based evaluation
            evaluation_prompt = (
                "You are an evaluation assistant that inspects two answers to determine if they match or not.\n\n"
                "Instructions:\n"
                "1. Compare the 'Correct Answer' and the 'Rationale'.\n"
                "2. If the Correct Answer matches the rationale's calculations/final answer, respond with the single word: correct\n"
                "3. If the Correct Answer is not given in the Rationale, respond with the single word: incorrect\n\n"
                f"Rationale:\n{generated_rationale}\n\n"
                f"Correct Answer:\n{correct_answer}"
            )

            # Interact with Ollama to evaluate the rationale
            response = ollama.chat(model="llama3.1:8b", messages=[
                    {
                        'role': 'user',
                        'content': evaluation_prompt
                    },
                ])

            decision = response['message']['content'].strip().lower()
            print(f"Decision: {decision}")
            
            if decision == "correct":
                # Double-check with programmatic evaluation
                    correct_entry = {
                        'question': question,
                        'rationale': generated_rationale,
                        'correct_answer': correct_answer,
                    }
                    correct_pairs.append(correct_entry)
                    print(f"Pair {idx} marked as correct.")
                    pair_correct = True  # Exit the retry loop
                    

            elif decision == "incorrect":
                    if attempts >= max_attempts:
                        incorrect_entry = {
                            'question': question,
                            'rationale': generated_rationale,
                            'correct_answer': correct_answer,
                        }
                        print(f"Reached maximum attempts for Pair {idx}. Adding to incorrect pairs.")
                        unmatched_pairs.append(incorrect_entry)
                    else:
                        print(f"Retrying Pair {idx} ({attempts}/{max_attempts})...")
            else:
                # Unexpected response; add to unanswered_pairs for manual review
                print(f"Unexpected decision '{decision}' for Pair {idx}. Adding to unanswered_pairs.")
                unanswered_entry = {
                            'question': question,
                            'rationale': generated_rationale,
                            'correct_answer': correct_answer,
                        }
                unanswered_pairs.append(unanswered_entry)
                break  # Exit the retry loop

    print("\nProcessing complete.")
    print(f"Correct Pairs: {len(correct_pairs)}")
    print(f"Incorrect Pairs: {len(unmatched_pairs)}")
    print(f"Unanswered Pairs: {len(unanswered_pairs)}")
    
def save_results(data, file_path):
    """
    Saves categorized pairs to a specified JSON file.

    Args:
        data (list): List of categorized pairs.
        file_path (str): Path to the output JSON file.
    """
    if not data:
        print(f"No data to save for {file_path}.")
        return

    # Check if the file exists; if not, create it with an empty list
    if not os.path.exists(file_path):
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump([], file, ensure_ascii=False, indent=4)

    # Load existing data
    with open(file_path, 'r', encoding='utf-8') as file:
        try:
            existing_data = json.load(file)
            if not isinstance(existing_data, list):
                print(f"Unexpected data format in {file_path}. Overwriting with a new list.")
                existing_data = []
        except json.JSONDecodeError:
            print(f"Error decoding JSON from {file_path}. Overwriting with a new list.")
            existing_data = []

    # Append new data
    existing_data.extend(data)

    # Save back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

    print(f"Saved {len(data)} pair(s) to {file_path}.")

# Run the processing function
process_incorrect_pairs(NUMBER_OF_PAIRS_TO_PROCESS, model_name=OLLOMA_MODEL_NAME)

Processing 4 out of 59 incorrect pair(s).

Processing Pair 1:
Question: Evaluate:60 - 12*3*2 =?
Correct Answer: 52
Attempt 1 for Pair 1.
Decision: incorrect
Reached maximum attempts for Pair 1. Adding to incorrect pairs.

Processing Pair 2:
Question: The length of rectangle is thrice its breadth and its perimeter is 56 m, find the area of the rectangle?
Correct Answer: 252
Attempt 1 for Pair 2.
Decision: incorrect
Reached maximum attempts for Pair 2. Adding to incorrect pairs.

Processing Pair 3:
Question: During a trip on an expressway, Don drove a total of x miles. His average speed on a certain 5-mile section of the expressway was 30 miles per hour, and his average speed for the remainder of the trip was 40 miles per hour. His travel time for the x-mile trip was what percent greater than it would have been if he had traveled at a constant rate of 60 miles per hour for the entire trip?
Correct Answer: 400/x%
Attempt 1 for Pair 3.
Decision: correct
Pair 3 marked as correct.

Processin

In [3]:
correct_pairs

[{'question': 'During a trip on an expressway, Don drove a total of x miles. His average speed on a certain 5-mile section of the expressway was 30 miles per hour, and his average speed for the remainder of the trip was 40 miles per hour. His travel time for the x-mile trip was what percent greater than it would have been if he had traveled at a constant rate of 60 miles per hour for the entire trip?',
  'rationale': "1. Don drove a total of x miles, with an average speed of 30 mph on a certain 5-mile section and 40 mph for the remainder.\n\n2. We can calculate his travel time as (total distance / average speed) = (x / (0.5 * 30 + 0.5 * 40)) hours.\n\n3. If he had traveled at a constant rate of 60 mph for the entire trip, his travel time would be x/60 hours.\n\n4. To find the percentage by which his actual travel time exceeds this ideal travel time, we can calculate ((actual travel time - ideal travel time) / ideal travel time) * 100%.\n\n5. Substituting our earlier calculations into t

In [4]:
unmatched_pairs

[{'question': 'Evaluate:60 - 12*3*2 =?',
  'rationale': "1. First, we need to simplify the expression inside the parentheses:\n12 * 3 * 2 = 12 * 6 = 72\n\n2. Now, let's substitute the simplified value back into the original equation:\n60 - 72 = ?\n\n3. To solve for , we'll subtract 72 from 60:\n\n60 - 72 = -12\nA: However, this is not one of the answer choices provided in the question. Therefore, there must be an error somewhere in our calculation.\n\n4. Let's review step #1 and notice that we multiplied three numbers together (12, 3, and 2) without considering their order of operations. According to PEMDAS (Parentheses, Exponents, Multiplication/Division, Addition/Subtraction), multiplication has equal precedence with division. Therefore, we need to simplify the expression inside the parentheses correctly:\n\n12 * (3 * 2) = 12 * 6 = 72\n\n5. Now that we have corrected step #1 and obtained a valid result of -12 for , let's substitute this value back into our original equation:\n\n60 - 

In [18]:
unanswered_pairs

[]

In [21]:
from src.data_appending import convert_correct_pairs_to_conversations, append_conversations_to_jsonl

new_conversations = convert_correct_pairs_to_conversations(correct_pairs)
    
# Append to the new JSONL file
append_conversations_to_jsonl(new_conversations, './data/finetuning_data_new.jsonl')

Successfully appended 1 conversations to './data/finetuning_data_new.jsonl'.


In [22]:
import json
import re

# Define the input and output file paths
input_jsonl = "data/finetuning_data_new.jsonl"       # Replace with your actual input JSONL file path
output_json = "data/formatted_data.json"    # Desired output JSON file path

# Initialize a list to hold reformatted entries
reformatted_entries = []

# Define the instruction text
instruction_text = "Provide a detailed answer to the following question."

def parse_q_a(text):
    """
    Parses the input text to extract Question and Answer.

    Args:
        text (str): The input text containing Q and A.

    Returns:
        tuple: (question, answer) if both are found, else (None, None).
    """
    lines = text.strip().split('\n')

    question_lines = []
    answer_lines = []

    current_section = None

    for line in lines:
        line = line.strip()
        if line.startswith('Q:'):
            current_section = 'question'
            question_lines.append(line[2:].strip())
        elif line.startswith('A:'):
            current_section = 'answer'
            answer_lines.append(line[2:].strip())
        else:
            if current_section == 'question':
                question_lines.append(line)
            elif current_section == 'answer':
                answer_lines.append(line)

    question = '\n'.join(question_lines).strip() if question_lines else None
    answer = '\n'.join(answer_lines).strip() if answer_lines else None

    return question, answer

# Open and read the input JSONL file
with open(input_jsonl, 'r', encoding='utf-8') as fin:
    for idx, line in enumerate(fin, 1):
        try:
            data = json.loads(line)
            text = data.get('text', '').strip()

            if not text:
                print(f"Warning: Empty 'text' field in line {idx}. Skipping.")
                continue

            # Parse the Question and Answer
            question, answer = parse_q_a(text)

            if not question or not answer:
                print(f"Warning: Missing Question or Answer in line {idx}. Skipping.")
                continue

            # Append the reformatted entry
            reformatted_entries.append({
                "instruction": instruction_text,
                "input": question,
                "output": answer
            })

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in line {idx}: {e}. Skipping.")
            continue

# Assemble the entries into a dictionary with 'train' split
dataset_dict = {
    "train": reformatted_entries
}

# Save the dictionary as a single JSON file
with open(output_json, 'w', encoding='utf-8') as fout:
    json.dump(dataset_dict, fout, ensure_ascii=False, indent=4)

print(f"Reformatting complete. {len(reformatted_entries)} entries saved to '{output_json}'.")

Error decoding JSON in line 308: Expecting ',' delimiter: line 1 column 2649 (char 2648). Skipping.
Reformatting complete. 315 entries saved to 'data/formatted_data.json'.
